In [34]:
import os
import requests
from tqdm import tqdm
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

In [37]:
options = Options()
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--window-size=1200,600')
driver= webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

url = 'https://search.shopping.naver.com/search/all?adQuery=%EC%82%AC%EB%A3%8C&frm=NVSHATC&origQuery=%EC%82%AC%EB%A3%8C&pagingIndex=1&pagingSize=40&productSet=checkout&query=%EC%82%AC%EB%A3%8C&sort=rel&timestamp=&viewType=image'

driver.get(url)
driver.implicitly_wait(5)
driver.execute_script("window.scrollBy(0, document.body.scrollHeight)")
driver.implicitly_wait(5)

product_pages = driver.find_elements(By.CSS_SELECTOR, 'div.thumbnail_thumb_wrap__RbcYO._wrapper > a')

for product_page in tqdm(product_pages, total=len(product_pages), desc='Processing Product Pages'):
    try:
        initial_tab_count = len(driver.window_handles)

        product_page.click()
        driver.implicitly_wait(5)

        new_window = driver.window_handles[-1]
        driver.switch_to.window(new_window)
        driver.implicitly_wait(5)
        
        driver.find_element(By.CSS_SELECTOR, 'div._3osy73V_eD._1Hc_ju_IXp > button').click()
        driver.implicitly_wait(5)

        driver.execute_script("window.scrollBy(0, document.body.scrollHeight)")
        driver.implicitly_wait(5)

        html_content = driver.page_source

        soup = BeautifulSoup(html_content, 'html.parser')

        product_name = soup.select_one('._22kNQuEXmb._copyable').text
        
        image_tags = soup.select('.se-main-container img')

        target_directory = f'/naver/{product_name}'

        if not os.path.exists(target_directory):
            os.makedirs(target_directory)

        for i, img in tqdm(enumerate(image_tags), total=len(image_tags), desc='Processing Images'):
            if 'data-src' in img.attrs and img['data-src'].strip():
                image_url = img['data-src']
                
                file_path = os.path.join(target_directory, f'image_{i}.jpg')

                response = requests.get(image_url)

                with open(file_path, 'wb') as f:
                    f.write(response.content)
        
        driver.close()
        
    except:
        print(product_page)
        pass

    driver.switch_to.window(driver.window_handles[0])

driver.quit()

Processing Product Pages:  58%|█████▊    | 26/45 [06:36<04:49, 15.24s/it]


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"div._3osy73V_eD._1Hc_ju_IXp > button"}
  (Session info: chrome=120.0.6099.217); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00EF6EE3+174339]
	(No symbol) [0x00E20A51]
	(No symbol) [0x00B36FF6]
	(No symbol) [0x00B69876]
	(No symbol) [0x00B69C2C]
	(No symbol) [0x00B9BD42]
	(No symbol) [0x00B87054]
	(No symbol) [0x00B9A104]
	(No symbol) [0x00B86DA6]
	(No symbol) [0x00B61034]
	(No symbol) [0x00B61F8D]
	GetHandleVerifier [0x00F94B1C+820540]
	sqlite3_dbdata_init [0x010553EE+653550]
	sqlite3_dbdata_init [0x01054E09+652041]
	sqlite3_dbdata_init [0x010497CC+605388]
	sqlite3_dbdata_init [0x01055D9B+656027]
	(No symbol) [0x00E2FE6C]
	(No symbol) [0x00E283B8]
	(No symbol) [0x00E284DD]
	(No symbol) [0x00E15818]
	BaseThreadInitThunk [0x756D7BA9+25]
	RtlIni